In [ ]:
%load_ext autoreload
%autoreload 2


import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
from copy import deepcopy
from os.path import join as pjoin


if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    
#EMG_dir = data_dir
EMG_dir = pjoin(data_dir, 'EMG')
rawname_ = 'S001_OFF_01'

#rawname_ = 'S99_off_move'
    
#rawname_ = 'S01_off_hold'
#rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_on_move'

rawname_ = 'S02_off_hold'
#rawname_ = 'S02_off_move'
#rawname_ = 'S02_on_hold'
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_hold'
#rawname_ = 'S03_off_move'

#rawname_ = 'S04_off_hold'
#rawname_ = 'S04_off_move'
#rawname_ = 'S04_on_hold'
#rawname_ = 'S04_on_move'

# rawname_ = 'S05_off_hold'
# rawname_ = 'S05_off_move'
# rawname_ = 'S05_on_hold'
# rawname_ = 'S05_on_move'

# rawname_ = 'S06_off_hold'
# rawname_ = 'S06_off_move'
# rawname_ = 'S06_on_hold'
# rawname_ = 'S06_on_move'

# rawname_ = 'S07_off_hold'
# rawname_ = 'S07_off_move'
# rawname_ = 'S07_on_hold'
# rawname_ = 'S07_on_move'

# rawname_ = 'S08_off_rest'
# rawname_ = 'S08_on_rest'

#rawname_ = 'S09_off_rest'

# rawname_ = 'S10_off_rest'
# rawname_ = 'S10_off_move'
import globvars as gv
raw,raw_rectconv,raw_env = None,None,None

rawname = rawname_ + '_resample_raw.fif'
rawname_rectconv = rawname_ + '_emg_rectconv.fif'
fname_full = os.path.join(gv.data_dir,rawname)
fname_rectconv_full = os.path.join(gv.data_dir,rawname_rectconv)
print(fname_rectconv_full)
if os.path.exists(fname_rectconv_full): 
    rectconvraw = mne.io.read_raw_fif(fname_rectconv_full, None)
    sfreq = int(rectconvraw.info['sfreq'])
elif os.path.exists(fname_full):
    # read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
    raw = mne.io.read_raw_fif(fname_full, None)
    sfreq = int(raw.info['sfreq'])
else:
    rawname_rectconv = rawname_ + '_anon_emg_rectconv.fif'
    fname_rectconv_full = os.path.join(EMG_dir,rawname_rectconv)
    rectconvraw = mne.io.read_raw_fif(fname_rectconv_full, None)
    
    rawname_env = rawname_ + '_anon_emg_rectconv_envelope.fif'
    fname_env_full = os.path.join(EMG_dir,rawname_env)
    raw_env = mne.io.read_raw_fif(fname_env_full, None)
    
    sfreq = int(rectconvraw.info['sfreq'])

In [ ]:
%matplotlib qt

In [ ]:
%matplotlib qt
raw_rectconv.plot(duration=25);

In [ ]:
# ax = plt.gca()
# ax.plot([1,2,3],[3,43,1])
# ax.plot([1,2,3],[5,3,34])
# print(ax.get_yticks() )
# print(ax.get_yticklabels()) 
# ax.set_yticklabels( len(ax.get_yticks()) * ['f'])

In [ ]:
HHUC_files_format = True
try:
    int( rawname_.split('_')[-1] )
except ValueError as e:
    print(str(e))
    HHUC_files_format = False

badchlist = None
maintremside = None
gen_subj_info = None
if not HHUC_files_format:
    # get info about bad MEG channels (from separate file)
    with open(pjoin(gv.code_dir,'subj_info.json') ) as info_json:
            #raise TypeError

        #json.dumps({'value': numpy.int64(42)}, default=convert)
        gen_subj_info = json.load(info_json)

    subj,medcond,task  = utils.getParamsFromRawname(rawname_)
    subj_num = int(subj[1:])
    if subj_num < 8:
        badchlist = gen_subj_info[subj]['bad_channels'][medcond][task]
    else:
        badchlist = gen_subj_info[subj]['bad_channels'][medcond]['hold']
        
    tremfreq = gen_subj_info[subj]['tremfreq']
    print('tremfreq = ',tremfreq)
        
    if raw is not None:
        raw.info['bads'] = badchlist
        print('bad channels are ',badchlist)


        skip_if_exist = 1

        import utils_preproc as upre
        tremfreq_for_preproc = tremfreq
        tremfreq_for_preproc = 10
        rectconvraw = upre.extractEMGData(raw, rawname_ = rawname_, 
                                          tremfreq=tremfreq_for_preproc, 
                                          skip_if_exist=skip_if_exist)
    else:
        sub_bads = [ chn for chn in badchlist if chn in rectconvraw.ch_names]
        rectconvraw.info['bads'] = sub_bads


    import gc; gc.collect()
else:
    rectconvraw = raw_rectconv

rectconvraw_perside = utils.getEMGperHand(rectconvraw)
if raw_env is not None:
    rectconvraw_env_perside = utils.getEMGperHand(raw_env)

if not rectconvraw.ch_names[0].endswith('_old'):
    rename_dict = dict( [(chn,chn+'_old') for chn in rectconvraw.ch_names] )
    rectconvraw.rename_channels(rename_dict)
print(rectconvraw.ch_names)



if gen_subj_info is not None:
    maintremside = gen_subj_info[subj]['tremor_side']
    nonmaintremside = utils.getOppositeSideStr(maintremside)
    mts_letter = maintremside[0].upper()
    ots_letter = utils.getOppositeSideStr(mts_letter)
    mts_trem_str = 'trem_{}'.format(mts_letter)
    mts_notrem_str = 'notrem_{}'.format(mts_letter)
    mts_task_str = '{}_{}'.format(task,mts_letter)
    ots_task_str = '{}_{}'.format(task,ots_letter)

In [ ]:
HHUC_files_format

## Collect info_times, save to one files and look at errors (exports from Hilbert)

In [ ]:
import subprocess as sp
c = 'ls /home/demitau/data/EMG/info_times_*.json'
out = sp.getoutput(c)
ss = '/home/demitau/data/EMG/'
fns = [fn[len(ss): ] for fn in out.split('\n') ]
import json

d = {}
for fn in fns:
    with(open(pjoin(ss,fn), 'r')) as f:
        jcur = json.load(f)
        d.update(jcur)

fn_full = pjoin(ss,'Info_times_all_with_trem.json')
with(open(fn_full, 'w')) as f:
    json.dump(d,f)

r = sp.getoutput(f'python -m json.tool  {fn_full}')
#rr = r.replace('\\n','\n')  # it breaks json:w

with(open(fn_full, 'w')) as f:
    f.write(r)

for k,s in d.items():
    for fn,subd in s.items():
        if 'ERROR' in subd:
            print(k,fn,subd['ERROR'][0])
            print(subd['ERROR'][1] )

In [ ]:
rectconvraw_perside['left'].plot(duration=60,scalings='auto')

In [ ]:
rectconvraw_env_perside['left'].plot(duration=60,scalings='auto')

In [ ]:
rectconvraw_perside['left'].plot_psd(fmax=10)
help (rectconvraw_perside['left'].plot_psd)

# Read and convert labels

In [ ]:
rawname_

In [ ]:
pwd

In [ ]:
#%debug
# for current raw

print(rawname_,'Main trem side ' ,maintremside,mts_letter)

anns_cnv, ivalis_perside = utils.intervalJSON2Anns(rawname_)
anns_cnv_Jan, ivalis_perside_Jan = utils.intervalJSON2Anns(rawname_, 
                                                           use_new_intervals=0)

## Load anns if they are there

In [ ]:
tremfreq

In [ ]:
anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
if os.path.exists(anns_fn_full):
    anns = mne.read_annotations(anns_fn_full)
    #raw.set_annotations(anns)
    anns_descr_to_remove = ['endseg', 'initseg', 'middle', 'incPost' ]
    anns_upd = utils.removeAnnsByDescr(anns, anns_descr_to_remove)

    anns_upd = utils.renameAnnDescr(anns, {'mvt':'hold', 'no_tremor':'notrem'})
else:
    print(anns_fn_full, ' does not exist')
    anns_upd = None



ann_dict = {'Jan':anns_cnv_Jan, 'prev_me':anns_cnv, 'new_me':anns_upd}

print('! {} Main trem side is {}, tremfreq={}\n'.format(rawname_, mts_letter,tremfreq) )

ann_len_dict = {}
meaningful_totlens = {}
for ann_name in ann_dict:
    print('{} interval lengths'.format(ann_name))
    #display(anns_cnv_Jan.description)
    anns = ann_dict[ann_name]
    if anns is None:
        continue
    
    if raw is not None:
        times = raw.times
    else:
        times = rectconvraw.times
    lens = utils.getIntervalsTotalLens(anns, True, times=times)
    display(lens  )
    #lens_cnv_Jan = utils.getIntervalsTotalLens(anns_cnv_Jan, True, times=raw.times)
    #display(lens_cnv_Jan  )
    if mts_trem_str not in anns.description:
        print('!! There is no tremor, accdording to {}'.format(ann_name))
        
    meaningul_label_totlen = lens.get(mts_trem_str,0) + lens.get(mts_task_str,0)
    meaningful_totlens[ann_name] = meaningul_label_totlen
    if meaningul_label_totlen < 10:
        print('Too few meaningful labels {}'.format(ann_name))
        
    for it in lens:
        if it.find(mts_task_str) < 0 and it.find(ots_task_str) >= 0:
            print('{} has task {} which is opposite side to tremor {}'.format(
                ann_name, ots_task_str, mts_task_str) )
        assert not( it.find(mts_task_str) >= 0 and it.find(ots_task_str) >= 0),\
            'task marked on both sides :('
            
        
    print('\n')
# print('\nmy prev interval lengths')
# lens_cnv = utils.getIntervalsTotalLens(anns_cnv, True, times=raw.times) 
# display(lens_cnv )
# #display(anns_cnv.description)
# if mts_trem_str not in anns_cnv.description:
#     print('!! There is no tremor, accdording to prev me')

In [ ]:
meaningful_totlens

In [ ]:
use_old_anns = 0
if meaningful_totlens.get('new_me',100) + meaningful_totlens['prev_me'] < 10:
    use_old_anns = 1
    print('Maybe we should use old anns')

In [ ]:
# note than maintremside should correspond to the rawname!
#artif_anns = utils.artif2ann(artif.get(rawname_,{}) ,artif_nms.get(rawname_,{}), maintremside)
#artif_anns

In [ ]:
anns_cnv.description

In [ ]:
anns_descr_to_remove = ['endseg', 'initseg', 'middle', 'incPost', 'incPre' ]
anns_cnv = utils.removeAnnsByDescr(anns_cnv, anns_descr_to_remove)

In [ ]:
anns_cnv = utils.renameAnnDescr(anns_cnv, {'mvt':'hold', 'no_tremor':'notrem', 'unk_activity_full':'undef',
                                           'middle_full':'trem'})

In [ ]:
anns_cnv

 ## Observe converted anns

In [ ]:
print('! Main trem side is {}\n'.format(mts_letter) )

In [ ]:
rcr_oldann = rectconvraw.copy()
rcr_oldann.set_annotations(anns_cnv)
rcr_oldann_perside = utils.getEMGperHand(rcr_oldann)

%matplotlib qt
rcr_oldann_perside['right'].plot(duration=50);

In [ ]:
rcr_oldann_perside['left'].plot(duration=10);

In [ ]:
rcr_oldann_perside['right'].plot(duration=50);

In [ ]:
# WARNING!! This will overwrite the annotation file!
if use_old_anns:
    anns_fn = rawname_ + '_anns.txt'  # not csv!, otherwise origtime is under and then everything is bad when read
    anns_fn_full = os.path.join(data_dir, anns_fn)
    print('Saving prev me anns to ',anns_fn_full)
    anns_cnv.save(anns_fn_full)

## Set annotations

In [ ]:
print( mts_letter )
%matplotlib qt

In [ ]:
adjustment_only = 1
if adjustment_only:
    rectconvraw.set_annotations(anns_upd)
    rectconvraw_perside = utils.getEMGperHand(rectconvraw)

In [ ]:
%matplotlib qt

In [ ]:
if adjustment_only:
    dur = 25
    rectconvraw_perside['left'].plot(duration=dur, scalings='auto');

In [ ]:
#help(%matplotlib )

In [ ]:
%matplotlib qt

In [ ]:
#start = 443; dur = 20
start = 129; dur = 35
rectconvraw_perside['left'].plot(duration=dur, scalings=3e-4, start=start );
plt.savefig(pjoin(gv.code_dir,'output',f'{rawname_}_{start}:{dur}.svg')  )

In [ ]:
if adjustment_only:
    rectconvraw_perside['right'].plot(duration=50);

In [ ]:
if adjustment_only:
    anns_fn = rawname_ + '_anns.txt'  # not csv!, otherwise origtime is under and then everything is bad when read
    anns_fn_full = os.path.join(data_dir, anns_fn)
    print('Saving prev me anns to ',anns_fn_full)
    raw.annotations.save(anns_fn_full)

In [ ]:
# chis = mne.pick_channels_regexp(raw.ch_names, 'EMG.*old')
# restr_names = np.array( raw.ch_names )[chis]

# raw.info['bads'] = []



# emgonly = raw.copy()
# emgonly.load_data()
# emgonly.pick_channels(restr_names.tolist())
# emgonly_unfilt = emgonly.copy()
# print(emgonly.ch_names)
# #help(emgonly.filter)

# y = {}
# for chname in emgonly.ch_names:
#     y[chname] = 'eeg'
# emgonly.set_channel_types(y)

# emgonly.filter(l_freq=10, h_freq=None, picks='all')

# maxtremfreq = 9
# windowsz = int( sfreq / maxtremfreq ) 
# print( 'wind size is {} s = {} bins'.format(windowsz/emgonly.info['sfreq'], windowsz ))

# rectconvraw = emgonly.copy()
# #hilbraw.plot(duration=2)

# rectconvraw.apply_function( np.abs)
# rectconvraw.apply_function( lambda x: np.convolve(x,  np.ones(windowsz),  mode='same') )
# #rectconvraw.apply_function( lambda x: x / np.quantile(x,0.75) )

# rectconvraw.apply_function( lambda x: x / 100 ) # 100 is just empirical so that I don't have to scale the plot 

# Detect tremor from scratch

In [ ]:
import globvars as gv
gv.EMG_per_hand

if badchlist is not None:
    tt = [s.find('EMG') >= 0  for s in badchlist ]
    if sum( tt ) > 0:
        print('There are bad EMG channels {}'.format(np.where(tt)[0]))

print(rawname_, 'maintremside ',maintremside)
import utils_preproc as upre
#EMG_per_hand = {'right':['EMG061_old', 'EMG062_old'], 'left':['EMG063_old', 'EMG064_old' ] }
EMG_per_hand = gv.EMG_per_hand

# rectconvraw_perside = {} 
# for side in EMG_per_hand:
#     chns = EMG_per_hand[side]
#     tmp = rectconvraw.copy()
#     tmp.pick_channels(chns)
    
#     rectconvraw_perside[side] = tmp
    
# for side in EMG_per_hand:    
#     badstr = '_'+utils.getOppositeSideStr(side[0].upper())
#     print(badstr)
#     anns_upd = utils.removeAnnsByDescr(rectconvraw_perside[side].annotations, [badstr])
#     rectconvraw_perside[side].set_annotations(anns_upd)
    
#     #rectconvraw_perside[side] = tmp

In [ ]:
%matplotlib qt

In [ ]:
#rectconvraw_perside['left'].plot();

In [ ]:
#rectconvraw_perside['right'].plot();

In [ ]:
#badchlist

# Automatic

In [ ]:
windowsz2 = int( 0.5 * sfreq )
print('{} maintremside = {}, tremfreq = {}'.format(rawname_,maintremside, tremfreq) )

### Select here which side we want to label 

In [ ]:
%matplotlib qt
use_maintremside = 0

if maintremside is not None:
    if use_maintremside:
        curside = maintremside
    else:
        curside = utils.getOppositeSideStr(maintremside)
    print('Working with maintremside ', curside == maintremside)
else:
    curside = 'right'

curraw = rectconvraw_perside[curside]
if raw is None:
    curraw_env = rectconvraw_env_perside[curside]
else:
    curraw_env = None

### Find quiet intervals automatic

In [ ]:
#plt.plot(rectconvraw.times,good)
#plt.axhline(qs[0])

In [ ]:
print(curraw.get_data().shape)

qval = 0.4
curdat = curraw.get_data()
qs = np.quantile( curraw.get_data(), qval, axis=1)[:,None]

good = np.prod( curdat <= qs, axis=0 )
print(np.sum(good))

good = good.astype(int)

cvl,pairs = utils.getIntervals(None, thr=0.1, cvl = good, extFactorR=-1.3,
                               extFactorL=-1.3, percentthr=0.8, 
                               minlen= 2 * sfreq, inc=1, min_dist_between= sfreq//2)
print('found {} intervals'.format(len(pairs)))

#---

newann = mne.Annotations([],[],[])
for stbin,endbin in pairs:
    curside_letter = curside[0].upper()
    descr = 'notrem_{}'.format(curside_letter)
    arg = {'onset':stbin/sfreq, 'duration':(endbin-stbin)/sfreq, 'description':descr}
    newann.append(**arg)
    
curraw.set_annotations(newann);
if curraw_env is not None:
    curraw_env.set_annotations(newann);

In [ ]:
#pairs = []

### Mark no tremor

In [ ]:
#help(curraw.plot)

In [ ]:
rawname_

In [ ]:
# they have to be in separate cells 
# because I may want to call plot sveral time to adjust annotations
curraw.plot(duration=60, scalings='auto');

In [ ]:
curraw_env.plot(duration=60, scalings='auto');

In [ ]:
deflabel = 'trem'
#deflabel = 'undef'
#deflabel = 'move'
task_wrong_side = 1

#### If we want to take 'notrem' from existing
use_exiting_notrem = 0
if use_exiting_notrem:
    ivalis = utils.ann2ivalDict(curraw.annotations)
    notrems = ivalis['notrem_{}'.format(curside[0].upper())]
else:
    anns = curraw.annotations
    notrems = [ (ann['onset'],ann['onset']+ann['duration'],'notrem_{}'.format(curside_letter)) for ann in anns]
    
# ---    

m, m0 = utils.getIntervalMaxs(curraw,notrems)
curraw_thr = curraw.copy()

rn = [(chn,chn + '_thr') for chn in curraw_thr.ch_names]
curraw_thr.rename_channels( dict(rn) )
print( curraw_thr.ch_names )

# convert data to its mask
curraw_thr.apply_function(lambda x:  np.heaviside(x - m[:,None] , 0)/100000 , 
                          channel_wise=0 )
#find_events(raw, consecutive=True, 
#...                   output='step')   # stime channel should be discerete
curraw_thr_display = curraw_thr.copy()
curraw_thr_display.add_channels([curraw])
#----------   create smoothed mask
curraw_thr_mav = curraw_thr.copy()

rn = [(chn,chn+'2') for chn in curraw_thr.ch_names]
curraw_thr_mav.rename_channels( dict(rn) )
print( curraw_thr_mav.ch_names )
curraw_thr_mav.apply_function( lambda x : np.convolve(x, np.ones(windowsz2), mode='same'))

# stats of the smoothened mask
m,m0 = utils.getIntervalMaxs(curraw_thr_mav, notrems)
print(m)
#-------

curraw_thr_mav.apply_function(lambda x:  np.heaviside(x - m[:,None] , 0)/100000 ,
                              channel_wise=0 )
#find_events(raw, consecutive=True, 
#...                   output='step')   # stime channel should be discerete
tmp = curraw_thr_mav.copy()
tmp.add_channels([curraw])

# prduct across (the two) channels
resd = np.prod(curraw_thr_mav.get_data(), axis=0)[None,:] #* 100000
resd /= np.max(resd)
print(np.max(resd))
# mask and smoothened 
resd = np.vstack( [ tmp.get_data(), resd])
#---------

newchn = 'EMG{}_thr2mav'.format(curside[0].upper() )
#newchn = 'STI'

info = mne.create_info(
    ch_names=tmp.ch_names + [newchn],
    ch_types=['eeg']*4 + ['stim'] ,
    sfreq= sfreq )

newr = mne.io.RawArray(resd,info)
newr.ch_names

curraw_thr_mav_display = newr
curraw_thr_mav_display.set_annotations(curraw_thr_mav.annotations)
#----------

# evts = mne.find_events(curraw_thr_mav_display, stim_channel=[newchn], output='step',
#                        consecutive=True, min_duration=3,shortest_event = 2)

tt,ts = curraw_thr_mav_display['stim']
cvl,pairs = utils.getIntervals(None, thr=0.1, cvl = tt[0], extFactorR=0,extFactorL=0, percentthr=0.8, 
                               minlen= 1 * sfreq, inc=1, min_dist_between= sfreq//2)
print('found {} intervals'.format(len(pairs)))

print('intervals (bin pairs) = ',pairs)

#----------
newann = mne.Annotations([],[],[])
for stbin,endbin in pairs:
    curside_letter = curside[0].upper()
    if stbin / sfreq <= 300 or ( task_wrong_side ):
        descr = '{}_{}'.format(deflabel,curside_letter)
        #print('fdff', stbin/sfreq)
    else:
        descr = '{}_{}'.format(task,curside_letter)
        #print('fd')
    arg = {'onset':stbin/sfreq, 'duration':(endbin-stbin)/sfreq, 'description':descr}
    newann.append(**arg)

#adding notrems
for st,end,descr in notrems:
    curside_letter = curside[0].upper()
    arg = {'onset':st, 'duration':(end-st), 'description':descr}
    newann.append(**arg)

newann.description

In [ ]:
print( maintremside, curside )
print(badchlist)

### set found anns to the raw

In [ ]:
# set found anns to the raw
nontrem_ann = mne.Annotations([],[],[])
curraw_thr_mav_display.set_annotations( newann + nontrem_ann)

In [ ]:
look_raw_emg = 0
if look_raw_emg:
    chis = mne.pick_channels_regexp(raw.ch_names, 'EMG.*old')
    restr_names = np.array( raw.ch_names )[chis]

    emgonly = raw.copy()
    emgonly.info['bads'] = []
    emgonly.load_data()
    emgonly.pick_channels(restr_names.tolist())
    emgonly_unfilt = emgonly.copy()
    print(emgonly.ch_names)
    #help(emgonly.filter)

    y = {}
    for chname in emgonly.ch_names:
        y[chname] = 'eeg'
    emgonly.set_channel_types(y)
    emgonly.set_annotations(curraw_thr_mav_display.annotations)
    #emgonly.filter(l_freq=10, h_freq=None, picks='all')
    emgonly.plot(duration=50);

In [ ]:
#curraw_thr_mav_display.set_annotations(emgonly.annotations)

In [ ]:
# Refine automatically set annotations
curraw_thr_mav_display.plot(duration=50);  #, events=evts)

In [ ]:
remove_BAD = 1
if remove_BAD:
    badstr = 'BAD_'#+utils.getOppositeSideStr(letter)
    print(badstr)
    anns_nobad = utils.removeAnnsByDescr(curraw_thr_mav_display.annotations, [badstr])
    curraw_thr_mav_display.set_annotations(anns_nobad)

In [ ]:
rectconvraw_perside2 = utils.getEMGperHand(rectconvraw)
for side in ['left', 'right']:
    rectconvraw_perside2[side].set_annotations( rectconvraw_perside[side].annotations )

In [ ]:
rectconvraw_perside2['left'].plot(duration=50);

In [ ]:
rectconvraw_perside2['right'].plot(duration=50);

In [ ]:
rectconvraw_perside = rectconvraw_perside2

In [ ]:
print('Main tremsdie = {},\nCurrent side =  {}'.format(maintremside, curside ))

### Set resulting annotations to the object

In [ ]:
print('Main tremsdie = {},\nCurrent side =  {}'.format(maintremside, curside ))

rectconvraw_perside[curside].set_annotations( curraw_thr_mav_display.annotations )

In [ ]:
# To check which sides are set already
for side in rectconvraw_perside:
    print(side,rectconvraw_perside[side].annotations)

In [ ]:
#curraw_thr_mav_display.annotations.description

In [ ]:
#curraw_thr_mav_display.set_annotations(anns_upd)

In [ ]:
#evts
#anns0 = mne.annotations_from_events(evts, sfreq=sfreq)
#anns0.duration

In [ ]:
#mne.find_events(curraw_thr_mav_display)

In [ ]:
rectconvraw_perside['right'].plot(duration=50);

In [ ]:
rectconvraw_perside['left'].plot(duration=50);

### clear anns from other side to avoid confusion

In [ ]:
#utils.renameAnnDescr(rectconvraw_perside['right'].annotations, {'BAD_':'undef_R'} )

In [ ]:
#a = utils.renameAnnDescr(rectconvraw_perside['right'].annotations, {'BAD_':'undef_R'} )
# rectconvraw_perside['right'].set_annotations(a)

In [ ]:
#curside = utils.getOppositeSideStr(maintremside )
# curside = 'right'
# letter = curside[0].upper() 
# badstr = 'BAD_'#+utils.getOppositeSideStr(letter)
# print(badstr)
# anns_upd = utils.removeAnnsByDescr(rectconvraw_perside[curside].annotations, [badstr])

In [ ]:
#rectconvraw_perside[curside].set_annotations(anns_upd)

In [ ]:
#%matplotlib qt
#rectconvraw_perside[curside].plot(duration=40)

## Merge two sides in one raw

In [ ]:
print(rawname_, maintremside, '\n')

#a = utils.renameAnnDescr(rectconvraw_perside['left'].annotations, {'notrem_R':'notrem_L', 'hold_R':'hold_L'})
#rectconvraw_perside['left'].set_annotations(a)

for side in rectconvraw_perside:    
    print(side, rectconvraw_perside[side].annotations.description )

In [ ]:
# merge
rectconvraw_upd = rectconvraw_perside['left'].copy()
rectconvraw_upd.add_channels([rectconvraw_perside['right']])
rectconvraw_upd.set_annotations( rectconvraw_perside['left'].annotations + rectconvraw_perside['right'].annotations )
rectconvraw_upd.annotations.description

# Check that everything is ok

In [ ]:
rectconvraw_upd.plot();

# Save

In [ ]:
anns = rectconvraw_upd.annotations

anns_fn = rawname_ + '_anns.txt'  # not csv!, otherwise origtime is under and then everything is bad when read
anns_fn_full = os.path.join(data_dir, anns_fn)
print(anns_fn_full)
anns.save(anns_fn_full)

In [ ]:
# descrs_upd = anns.description   # note that annotation can be more than 12 sym long!
# for i,descr in enumerate(descrs_upd):
#     #if descr != 'mvt_L' and descr != 'other_muscle':
#     #    descrs_upd[i] += '_L'
#     print(i,descrs_upd)
#     if descr == 'no_tremor_L':
#         descrs_upd[i] = 'trem_L'
#         print(descrs_upd[i])
#         descr+= '_R'